In [43]:
import codecs

class Recording:
    def __init__(self, id, type):
        self.id=id
        self.type = type
        value = 64*[255*[0]]
        
class Person:
    recordings = []
    def __init__(self, id):
        self.id = id
    def setRecordings(recordings):
        self.recordings = recordings



# FP1 chan 0

# FP2 chan 1

# F7 chan 2

# F8 chan 3

# AF1 chan 4

# AF2 chan 5

# FZ chan 6

# F4 chan 7

# F3 chan 8

# FC6 chan 9

# FC5 chan 10

# FC2 chan 11

# FC1 chan 12

# T8 chan 13

# T7 chan 14

# CZ chan 15

# C3 chan 16

# C4 chan 17

# CP5 chan 18

# CP6 chan 19

# CP1 chan 20

# CP2 chan 21

# P3 chan 22

# P4 chan 23

# PZ chan 24

# P8 chan 25

# P7 chan 26

# PO2 chan 27

# PO1 chan 28

# O2 chan 29

# O1 chan 30

# X chan 31

# AF7 chan 32

# AF8 chan 33

# F5 chan 34

# F6 chan 35

# FT7 chan 36

# FT8 chan 37

# FPZ chan 38

# FC4 chan 39

# FC3 chan 40

# C6 chan 41

# C5 chan 42

# F2 chan 43

# F1 chan 44

# TP8 chan 45

# TP7 chan 46

# AFZ chan 47

# CP3 chan 48

# CP4 chan 49

# P5 chan 50

# P6 chan 51

# C1 chan 52

# C2 chan 53

# PO7 chan 54

# PO8 chan 55

# FCZ chan 56

# POZ chan 57

# OZ chan 58

# P2 chan 59

# P1 chan 60

# CPZ chan 61

# nd chan 62

# Y chan 63

